In [55]:
from collections import defaultdict
import copy
import json

from json_extractor import from_file_get_n_docs

from tagged_document_parsing_lib import \
all_possible_mentions_for_surface_forms,\
generate_mentions_for_surface_form,\
flatten

from stanford_parser import parse_stanford_doc
from wikifier_parser import parse_wikifier_doc
from tagme_parser    import parse_tagme_doc

# Read in Data

In [61]:
stanford_file_name = './50_tagged_by_stanford.jsonl'
wikifier_file_name = './100_tagged_by_wikifier.jsonl'
tagme_file_name = './100_tagged_by_tagme_with_raw_longtext_0_epsilon_dot1_includecategories_includeallspots.jsonl'

In [58]:
stanford_docs = from_file_get_n_docs(stanford_file_name, 10)

In [59]:
wikifier_docs = from_file_get_n_docs(wikifier_file_name, 10)

In [60]:
tagme_docs = from_file_get_n_docs(tagme_file_name, 10)

# Parse Stanford
From here on, parsing means, converting the output from the stanford tagger into one dictionary with two main keys: entities and tagged-words.

Entities is a map with the entity surface_form and type in json form as a key, and a list of mentions in the doc as value.

tagged-words is a map with a token word index as key, and the mention in which surface_form it is contained as value.

In [47]:
stanford_parsed_docs = [parse_stanford_doc(doc) for doc in stanford_docs]

# Parse Wikifier

In [48]:
wikifier_parsed_docs = [parse_wikifier_doc(doc) for doc in wikifier_docs]

# Parse Tagme

In [62]:
from tagged_document_parsing_lib import generate_mentions_for_surface_form
import copy
from collections import defaultdict

def parse_tagme_doc(doc):
    indexed_tagme_annotations = defaultdict(lambda: [])
    parsed_doc = {}
    parsed_doc[u'tagged-words'] = {}
    
    raw_text = doc['raw']
    parsed_doc[u'tokens'] = raw_text.split()

    for i, annotation in enumerate(doc['annotations']):
        all_possible_annotation_indexes = generate_mentions_for_surface_form(annotation['spot'])

        surface_form = annotation['spot']
        
        start_word_index = raw_text[:annotation['start']].split().__len__()
        end_word_index = start_word_index + len(surface_form.split())

        annotation[u'start'] = start_word_index
        annotation[u'end'] = end_word_index - 1

        for index in all_possible_annotation_indexes:
            indexed_tagme_annotations[json.dumps(index, ensure_ascii=False)].append(annotation)
            
        for i in range(start_word_index, end_word_index):
            parsed_doc[u'tagged-words'][i] = copy.deepcopy(annotation)
            
    parsed_doc[u'entities'] = indexed_tagme_annotations
    
    return parsed_doc

In [63]:
tagme_parsed_docs = map(parse_tagme_doc, tagme_docs)

u'Andy Wilson'